# Projeto

**Objetivo**: Desenvolver um CRUD utilizando um arquivo JSON para "persistir" os dados e aplicar algumas outras funcionalidades que seriam interessantes no sistema.

**Requisitos**: O projeto deve conter as seguintes funcionalidades/ características:

- Utilizar JSON para persistir os dados
- Função de criar o arquivo JSON (amanda) OK
- Função para adicionar registros (através do *input*, ou de um conjunto de dados históricos, por exemplo) (ricardo)
    - Não deixar para o usuário decidir o id, apenas coloque o proximo valor disponível
- Função para realizar o *udpate* (atualização) das informações (leticia)
    - Selecionar pelo ID
- Função para excluir registros (adriely)
    - Selecionar pelo ID
- Função para realizar a leitura de registros (printar os dados na tela do consumidor x, lista) (leticia)
- Função para filtrar informações dos dados (Amanda) +/- OK
- Função para aplicar alguma regra de negócio (varia de acordo com o sistema, exemplo, grupo criou um sistema de consulta de colaboradores, logo deseja-se ter uma função que calcula o faturamento de todos os funcionários que são vendedores) (lorrany)
    - Utilizar as funções de filtro e de estatística
- Função que retorne alguma estatística dos dados (lorrany)
- menu (daniel)
- Exemplos de funcionamento de cada uma das funções acima

**Pontos importantes**:

- Organização de código no ``jupyter notebook`` (não precisa de ppt, no dia apenas o notebook será necessário)
- Definição sobre o que será o sistema (livre e definido pelo grupo)
- Apresentação do sistema e funcionalidades (assim como os exemplos). Não precisa ser o grupo todo apresentando
- Gerenciamento de erros, como no caso de num input o usuário colocar 'abc' num campo de valor inteiro
- A nota do grupo será a nota de cada integrante
- Tempo de apresentação: de 15 a 20 minutos
- A princípio todos integrantes devem entregar o mesmo notebook no sistema do LMS, para fins de retorno de nota
- Não precisa se ater somente aos 10 requisitos acima, pode ir além :)

banco de dados
https://www.kaggle.com/datasets/iamsouravbanerjee/customer-shopping-trends-dataset

## Função para a leitura de dados de um csv ou json e transformação em uma lista de dicionários

In [5]:
import json
import csv
def importar_dados(extensao:str):
    dados = list()
    
    opcoes = arquivos_disponiveis(extensao)
    respostas_validas = list(opcoes.keys())
    pergunta = 'Informe o numero do arquivo que deseja importar'
    str_arquivo = opcoes[validacao_respostas(pergunta, respostas_validas)]
    
    if extensao == '.json':
        with open(str_arquivo, "r", encoding = "utf-8") as f:
            dados = json.load(f)

    else:
        with open(str_arquivo, "r") as arquivo:
            arquivo_csv = csv.reader(arquivo, delimiter=",")
            next(arquivo_csv)  # pula a primeira linha (cabeçalho)
            for linha in enumerate(arquivo_csv):
                #print(linha)
                Customer_ID, Age, Gender, Item_Purchased,Category, Purchase_Amount, Location, Size, Color, Season, Review_Rating, Subscription_Status, Payment_Method, Shipping_Type, Discount_Applied, Promo_Code_Used, Previous_Purchases, Preferred_Payment_Method, Frequency_of_Purchases = linha[1]
                #print('ID', Customer_ID,'Idade', Age,'Genero', Gender,'Item_Comprado',Item_Purchased,
                #                          'Categoria', Category,'Metodo_Pagamento', Payment_Method)
                # 
                dados.append({'ID': int(Customer_ID), #numérica
                                    'Idade': int(Age), #numérica
                                    'Genero': Gender, #categórica
                                    'Item_Comprado':Item_Purchased, #categórica
                                    'Categoria': Category, #categórica
                                    'Preco': float(Purchase_Amount), #numérica
                                    'Metodo_Pagamento': Payment_Method, #categórica
                                    'Quantidade_Compras_Anteriores': int(Previous_Purchases), #numérica
                                    'Frequencia_Compra': Frequency_of_Purchases #categórica
                                    })
    return dados
    

# Traduzindo os valores retirados do CSV

In [2]:
def traduzindo_lista_dict(lista: list):
    for chave in list(lista[0].keys()):
        if type(lista_vendas[0][chave]) == str:
            categorias = lista_de_valores_sem_repeticoes(lista, chave)
            dict_traducao = traduz_palavras(categorias)
            for i in range(len(lista)):
                lista[i][chave] = dict_traducao[lista[i][chave]]
    return lista

## Função para traduzir palavras

In [3]:

from translate import Translator
def traduz_palavras(lista_de_palavras):
  """
  Recebe uma lista de palavras em inglês e retorna um dicionário onde 
  a chave é a palavra da lista e o valor é a tradução em português.

  Args:
    lista_de_palavras: Uma lista de palavras em inglês.

  Returns:
    Um dicionário.
  """
  dicionario_de_traducao = {}
  for palavra in lista_de_palavras:
    traducao = Translator(to_lang='pt-br', from_lang='english')
    dicionario_de_traducao[palavra] = traducao.translate(palavra)
  return dicionario_de_traducao


ModuleNotFoundError: No module named 'translate'

# Função que retorna uma lista de valores sem repetição

In [ ]:
def lista_de_valores_sem_repeticoes(lista_de_dicionarios, chave):
  """
  Recebe uma lista de dicionários.
  A função retorna lista dos valores encontrados para a chave, sem repetições de valores.

  Args:
    lista_de_dicionarios: Uma lista de dicionários.
    chave: A chave a ser pesquisada.

  Returns:
    Uma lista.
  """
  lista_de_valores = set()
  for dicionario in lista_de_dicionarios:
    if chave in dicionario:
      lista_de_valores.add(dicionario[chave])
  return list(lista_de_valores)

# Função que lista os arquivos disponíveis

In [ ]:
import os
def arquivos_disponiveis(extensao: str):
    pergunta = 'Informe o numero do arquivo que deseja importar'
    opcoes = dict()
    arquivos = os.listdir() # lista todos os arquivos presentes na pasta
    arquivos_extensao = [arquivo for arquivo in arquivos if arquivo.endswith(extensao)] # filtra pela extensao
    print(f'Lista de arquivos {extensao.upper()} disponíveis')
    for i in range(len(arquivos_extensao)):
        print(f'\t{i+1}\t{arquivos_extensao[i]}')
        opcoes.update({str(i+1):arquivos_extensao[i]})
    return opcoes
        

## Função de filtro

In [ ]:
def filtro(lista: list, categoria:str, valor_categoria, tipo:str):
    if tipo == 'Categorico':
        filtro = [ i for i in lista if i[categoria] == valor_categoria]
    if tipo == 'Maior que':
        filtro = [ i for i in lista if i[categoria] > valor_categoria]
    if tipo == 'Maior e iqual a':
        filtro = [ i for i in lista if i[categoria] >= valor_categoria]
    if tipo == 'Menor que':
        filtro = [ i for i in lista if i[categoria] < valor_categoria]
    if tipo == 'Menor e iqual a':
        filtro = [ i for i in lista if i[categoria] <= valor_categoria]
    if tipo == 'Igual a':
        filtro = [ i for i in lista if i[categoria] == valor_categoria]
    return filtro
    
    
dados_filtrados = filtro(dados_traduzidos, 'Idade',30,'Maior e iqual a')

## Função para validação de respostas do usuário

In [ ]:
def validacao_respostas(pergunta, respostas_validas):
  """
  Validação de respostas. A função valida respostas de um caracter
  É passado como parâmetro:
    - A pergunta que se quer fazer (str)
    - Uma lista com as respostas válidas

  Caso o usuário digite algo diferente das respostas válidas,
  a função repetirá a pergunta até que o usuário responda um valor válido.
  E só retornará opções válidas de valores
  """
  print(pergunta)
  resp = input().upper()[0]# apenas 1ªletra
  pergunta = 'Resposta inválida! ' + pergunta
  while not resp in respostas_validas:
    print(pergunta)
    resp = input().upper()[0]# apenas 1ªletra
    #resp = input(pergunta).upper()[0]# apenas 1ªletra
  return resp

## Função para exportar uma lista de dicionários para um arquivo .json

In [ ]:
import json
import os.path
def exportar_json(lista_vendas: list):
    """
    A função exporta a lista_vendas para um arquivo.
    O arquivo será nomeado pelo usuário
        Caso o arquivo já exista na pasta, o usuário será questionado se quer sobrescrever.
    O arquivo será no formato .json
    Será importado na mesma pasta que se encontra o script
    Args:
        lista_vendas (list): lista de dicionários com as informações de venda
    """
    nome_arquivo = input("Informe o nome do arquivo a ser criado:\t").lower()
    

    # retirando os possíveis espaços do nome
    qtd_espaco = nome_arquivo.count(' ')
    if qtd_espaco > 0:
        nome_arquivo =  nome_arquivo.replace(" ", "_", qtd_espaco)
    nome_arquivo += '.json'
    
    if os.path.exists(nome_arquivo) == True:
        pergunta = 'Já existe um arquivo com esse nome.\nDeseja sobrescrever?[S]  Sim\t [N] Não\t'
        resp_sobrescrever = validacao_respostas(pergunta, ['S','N'])
        if resp_sobrescrever == 'N':
            print('O arquivo não foi modificado.')
            return
    try:
        json_vendas = json.dumps(lista_vendas, indent=2,ensure_ascii=False)
        arquivo = open(nome_arquivo, 'w', encoding = "utf-8") 
        arquivo.write(json_vendas) 
        arquivo.close()
        print("Arquivo exportado com sucesso!")
    except:
        print("Erro ao exportar arquivo")
    return

# Menu filtros

In [ ]:
def opcoes_filtros(dados:list):
    tamanho = len(list(dados[0].keys())[1:])
    lista_categorias = list(dados[0].keys())[1:]
    respostas_validas = list(range(1,tamanho+1))
    respostas_validas = list(map(str, respostas_validas))

    print('\nCategorias Disponíveis para consulta:')
    for i in range(tamanho):
        print(1+i,'\t',lista_categorias[i])
    pergunta = 'Informe o número da categoria que deseja filtrar:'
    resp = validacao_respostas(pergunta, respostas_validas)
    chave = lista_categorias[int(resp)-1]
    if type(dados[0][chave]) == str:
        pergunta = f'Informe o número da opção voce quer filtrar dentro de {chave}: '
        opcoes = lista_de_valores_sem_repeticoes(dados, chave)
        respostas_validas = list(range(1,len(opcoes)))
        respostas_validas = list(map(str, respostas_validas))
        print(f'\nLista de opções dentro da categoria {chave}')
        for i in range(len(opcoes)):
            print(1+i,'\t',opcoes[i])
        resp = validacao_respostas(pergunta, respostas_validas)
        opcao = opcoes[int(resp)-1]
        print(filtro(dados, chave, opcao, 'Categorico'))
    else: # numeros reais ou inteiros
        pass

## Detectar idioma

In [ ]:
from langdetect import detect 
def detectar_idioma(dados:list):
    if detect(dados[0]['Metodo_Pagamento']) == 'en':
        pergunta = 'Os dados estão em inglês, deseja traduzir?[S] Sim  [N] Não\t'
        resp = validacao_respostas(pergunta, ['S','N'])
        if resp == 'S':
            print("Essa operação pode demorar, aguarde.")
            dados = traduzindo_lista_dict(dados)
    return dados


# Função Principal

In [ ]:
def main():
    
    pergunta = """Que tipo de arquivo deseja importar:
      1 \t CSV
      2 \t JSON
      3 \t Não importar nada   """
    resp =  validacao_respostas(pergunta, ['1','2','3'])
    if   resp == '1':
        dados = importar_dados('.csv')
        detectar_idioma(dados)
    elif resp == '2':
        dados = importar_dados('.json')
        detectar_idioma(dados)
    else:
        dados = list()
        
    continua = 'S'
    while continua =='S':
        pergunta = """Qual ação deseja realizar?
        Para adicionar um registro             --- A
        Para remover um registro               --- R
        Para editar informações                --- E
        Para leitura de registro               --- L
        Para mostrar a regra de negócio        --- N
        Para realizar estatística              --- S
        Para realizar filtro                   --- F
        """
        opcao = validacao_respostas(pergunta, ['A','R','E','L','N','S','F'])
        if   opcao == 'A': #ricardo
            pass
        elif opcao == 'R': #leticia
            pass
        elif opcao == 'E': #adriely
            pass
        elif opcao == 'L': #leticia
            pass
        elif opcao == 'N': #lorrany
            pass
        elif opcao == 'S': #lorrany
            pass
        else: #filtro #amanda
            pass
        
        pergunta = 'Deseja continuar no menu? [S]/[N]\t'
        continua = validacao_respostas(pergunta, ['S', 'N'])
        
main()
    

# Estatísticas

## Função para calcular o Ticket Médio

In [22]:
def calcular_ticket_medio(dados):
    """
    O ticket médio é um indicador usado para avaliar o gasto médio dos clientes.
    Para calculá-lo, deve-se dividir o faturamento total das vendas pelo número de vendas do intervalo de tempo em questão.
    """
    faturamento = 0
    for item in dados:
        faturamento = sum([faturamento, item["Preco"]])
    
    transacoes = len(dados)

    if transacoes == 0:
        # Para evitar divisões por zero
        return 0
    else:
        ticket_medio = faturamento / transacoes
        return ticket_medio

In [24]:
dados1 = [
    {
        "ID": 1,
        "Idade": 55,
        "Genero": "Homem",
        "Item_Comprado": "Blusa",
        "Categoria": "Vestuário",
        "Preco": 53.0,
        "Metodo_Pagamento": "Cartão de Crédito",
        "Quantidade_Compras_Anteriores": 14,
        "Frequencia_Compra": "Quinzenalmente"
    },
    {
        "ID": 2,
        "Idade": 19,
        "Genero": "Homem",
        "Item_Comprado": "Agasalho",
        "Categoria": "Vestuário",
        "Preco": 64.0,
        "Metodo_Pagamento": "Transferência Bancária",
        "Quantidade_Compras_Anteriores": 2,
        "Frequencia_Compra": "Quinzenalmente"
    },
    {
        "ID": 3,
        "Idade": 50,
        "Genero": "Homem",
        "Item_Comprado": "Jeans",
        "Categoria": "Vestuário",
        "Preco": 73.0,
        "Metodo_Pagamento": "dinheiro",
        "Quantidade_Compras_Anteriores": 23,
        "Frequencia_Compra": "Toda semana"
    }
]

calcular_ticket_medio(dados1)

'O ticket médio é: R$63.33'

# Regra de negócio

In [ ]:
def grupo_clientes(dados):
    """
    A clusterização de clientes permite agrupá-los por alguma semelhança.
    É um meio para entender o perfil de consumo dos clientes para que, de forma estratégica,
      sejam criados ofertas personalizadas.
    Nessa função os clientes serão separados em dois grupos:
    - A: perfil de consumo a partir do ticket médio
    - B: consome abaixo do ticket médio
    """
    for itens in dados:
        if Preco >= calcular_ticket_medio(dados):
            adicionar_registro("Grupo" : "A")
        else:
            adicionar_registro("Grupo" : "B")